In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('dataset/전국어린이보호구역표준데이터-20210801.xls')
df.columns = df.iloc[0]
df = df.iloc[1:]


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/전국어린이보호구역표준데이터-20210801.xls'

In [ ]:
def seoul(x):
    import re

    p = re.compile('서울')

    r = p.findall(x)
    
    return ''.join(r)
df['제공기관명'] = df.제공기관명.apply(seoul)

df = df[(df['제공기관명']=='서울')]

In [ ]:
df1 = df[(df['소재지지번주소'].isnull()==True)].copy()
df1

In [78]:
import requests
import json
import numpy as np


addrs = list(df1['소재지도로명주소'])
locations = []


for addr in addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
        "Authorization" : "KakaoAK ec276bf4e2dcd1c5055b2da48ef1b6f7"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)
    
city = []
town = []

for i in range(len(locations)):
    
    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_name'))
        
    except IndexError:
        print(i,'번째 주소 못가져옴',end ='')
        print()
        city.append('없음')
        town.append('없음')
        
    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_name'))


city_town = np.array([city,town]).T
df_temp = pd.DataFrame(city_town, columns = ['2', '3'])
# asan['소재지지번주소'] = df_temp[['3']]
# asan['소재지지번주소'] = df_temp[['3']]

87 번째 주소 못가져옴
100 번째 주소 못가져옴
126 번째 주소 못가져옴


In [101]:
a = pd.DataFrame(city_town, columns = ['구','동'])
a['소재지지번주소'] = a[['구','동']].agg(' '.join,axis=1)

In [102]:
a.drop(['구','동'],axis=1,inplace=True)

In [147]:
len(list(a['소재지지번주소']))

431

In [148]:
df1['소재지지번주소'] = list(a['소재지지번주소'])

In [ ]:
df1.loc[8042,'소재지지번주소'] = '강남구 개포2동'
df1.loc[8055,'소재지지번주소'] = '강남구 개포2동'
df1.loc[8081,'소재지지번주소'] = '도봉구 도봉1동'

In [178]:
df1.to_csv('dataset/서울_어린이보호구역_주소O.csv')

a = pd.read_csv('dataset/서울_어린이보호구역_주소O.csv')